# 数据读取以及说明

In [290]:
import pandas as pd
data=pd.read_csv('D:\\Users\\Administrator\\Desktop\\jinrong2\\application.csv')
pd.set_option('display.max_columns',None)
data.head(1)

,member_id,loan_amnt,term,loan_status,int_rate,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,pub_rec_bankruptcies,issue_d,earliest_cr_line
0,1,5000,36 months,Fully Paid,10.65%,10+ years,RENT,24000.0,Verified,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0,1,NaN,NaN,3,0,9,0.0,Dec-11,Jan-85


In [291]:
data.describe()

,member_id,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,pub_rec_bankruptcies
count,39785.000000,39785.000000,3.978500e+04,39785.000000,39785.000000,39785.000000,14058.000000,2791.000000,39785.000000,39785.000000,39785.000000,39088.000000
mean,19893.043861,11231.391228,6.897964e+04,13.317718,0.146513,0.869046,35.903329,69.747402,9.293628,0.055146,22.089959,0.043287
std,11485.157206,7464.634092,6.376333e+04,6.678367,0.491814,1.070083,22.018039,43.805070,4.399348,0.237656,11.401550,0.204383
min,1.000000,500.000000,4.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000
25%,9947.000000,5500.000000,4.050000e+04,8.180000,0.000000,0.000000,18.000000,22.500000,6.000000,0.000000,13.000000,0.000000
50%,19893.000000,10000.000000,5.900000e+04,13.410000,0.000000,1.000000,34.000000,90.000000,9.000000,0.000000,20.000000,0.000000
75%,29839.000000,15000.000000,8.235000e+04,18.600000,0.000000,1.000000,52.000000,104.000000,12.000000,0.000000,29.000000,0.000000
max,39786.000000,35000.000000,6.000000e+06,29.990000,11.000000,8.000000,120.000000,129.000000,44.000000,4.000000,90.000000,2.000000


# 数据预处理
初步看到有几个字段的数据是无法直接用的,所以要对数据进行预处理
* term 清洗
* emp_lengt 清洗
* desc 字段信息太多了,这里也需要进行处理,(这里替换成缺失值与非缺失值最为一个特征)
* 日期需要转化成pandas能识别的datatime格式
* mths_since_last_delinq和mths_since_last_record数据缺失值较多,需要进行处理
### 函数定义

In [368]:
import datetime 
import re
#term
def term_clean(x):
    x=str(x)
    return int(x.replace(' months',''))
#emp_lengt处理函数
def CareerYear(x):
    x=str(x)
    if x.find('nan') > -1:
        return -1
    elif x.find("10+")>-1:
        return 11
    elif x.find('< 1') > -1:
        return 0
    else:
        return int(re.sub("\D", "", x))
#desc处理函数
def DescExisting(x):
    x=str(x)
    if x=='nan':
        return 'no desc'
    else:
        return 'desc'
#日期处理函数
def ConvertDateStr(x,format):
    if str(x) == 'nan':
        return datetime.strptime('9900-1','%Y-%m')
    else:
        return datetime.strptime(x,format)
#缺失值
def MakeupMissing(x):
    if np.nan(x):
        return -1
    else:
        return x


# 开始清洗数据

In [381]:
#term
data['term']=data['term'].map(term_clean)
#loan_status 目标,Charged Off是违约用户用1表示
data['y']=data['loan_status'].map(lambda x:int(x=='Charged Off'))
#由于存在不同的贷款期限（term），申请评分卡模型评估的违约概率必须要在统一的期限中，且不宜太长，所以选取term＝36months的行本
data=data[data.term==36]
#将数据分为测试集和训练集
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(data,test_size=0.4)

In [384]:
#int_rate
data[]

,member_id,loan_amnt,term,loan_status,int_rate,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,pub_rec_bankruptcies,issue_d,earliest_cr_line,emp_length1,y
0,1,5000,36,Fully Paid,10.65%,10+ years,RENT,24000.00,Verified,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0,1,NaN,NaN,3,0,9,0.0,Dec-11,Jan-85,11,0
2,3,2400,36,Fully Paid,15.96%,10+ years,RENT,12252.00,Not Verified,NaN,small_business,real estate business,606xx,IL,8.72,0,2,NaN,NaN,2,0,10,0.0,Dec-11,Nov-01,11,0
3,4,10000,36,Fully Paid,13.49%,10+ years,RENT,49200.00,Source Verified,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.<br>",other,personel,917xx,CA,20.00,0,1,35.0,NaN,10,0,37,0.0,Dec-11,Feb-96,11,0
5,6,5000,36,Fully Paid,7.90%,3 years,RENT,36000.00,Source Verified,NaN,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0,3,NaN,NaN,9,0,12,0.0,Dec-11,Nov-04,3,0
7,8,3000,36,Fully Paid,18.64%,9 years,RENT,48000.00,Source Verified,Borrower added on 12/16/11 > Downpayment for a car.<br>,car,Car Downpayment,900xx,CA,5.35,0,2,NaN,NaN,4,0,4,0.0,Dec-11,Jan-07,9,0
11,12,12000,36,Fully Paid,12.69%,10+ years,OWN,75000.00,Source Verified,NaN,debt_consolidation,Consolidation,913xx,CA,10.78,0,0,NaN,NaN,12,0,34,0.0,Dec-11,Oct-89,11,0
12,13,9000,36,Charged Off,13.49%,< 1 year,RENT,30000.00,Source Verified,Borrower added on 12/15/11 > Plan to pay off 2 charge accounts. I will close one of them and ask for a credit line decrease from the other. Also borrowed money from a friend and would like to pay that off.......<br><br> Borrower added on 12/17/11 > The credit card that I am asking to be decreased will be ONLY for emergency purposes.....<br>,debt_consolidation,freedom,245xx,VA,10.08,0,1,NaN,NaN,4,0,9,0.0,Dec-11,Apr-04,0,1
13,14,3000,36,Fully Paid,9.91%,3 years,RENT,15000.00,Source Verified,"Borrower added on 12/19/11 > I intend to pay this debt off within half the time allotted.Home-brewer investing in equipment, hopefully someday a brewpub owner with delicious beers to sell and locally grown produced meat and vegetables.So, this is my start, and thank you for lending! One day I will repay the favor and lend back<br>",credit_card,citicard fund,606xx,IL,12.56,0,2,NaN,NaN,11,0,11,0.0,Dec-11,Jul-03,3,0
14,15,10000,36,Charged Off,10.65%,3 years,RENT,100000.00,Source Verified,NaN,other,Other Loan,951xx,CA,7.06,0,2,NaN,NaN,14,0,29,0.0,Dec-11,May-91,3,1
15,16,1000,36,Fully Paid,16.29%,< 1 year,RENT,28000.00,Not Verified,NaN,debt_consolidation,Debt Consolidation Loan,641xx,MO,20.31,0,1,NaN,NaN,11,0,23,0.0,Dec-11,Sep-07,0,0


datetime.datetime(9900, 1, 1, 0, 0)

datetime.datetime(2018, 1, 26, 14, 7, 45, 918459)